In [ ]:
import psana
import numpy as np
import holoviews as hv
from holoviews import dim
hv.extension('bokeh')
import panel as pn
import sys
sys.path.append('/sdf/group/lcls/ds/tools/smalldata_tools/latest/')
from smalldata_tools.utilities_plotting import hv_image_ctl
from smalldata_tools.utilities_plotting import hv_image

import scipy.ndimage as ndi

run = 631 
exp = 'xpptut15'
dsname = 'exp={}:run={}'.format(exp,run,exp[:3],exp)

detname = 'epix_alc1'
ds = psana.DataSource(dsname)
det = psana.Detector('epix_alc1')

ped = det.pedestals(run)
rms = det.rms(run)
gain = det.gain(run)

for i in range(2):
    event = next(ds.events())

# We will now compare different calibration options for the epix100
One is to use simple pedestal subtracted raw data, for the second option we add in the gain files determined by the detector group. In addition, we use two different methods to correct for common-mode noise which are described [here](https://confluence.slac.stanford.edu/display/PSDM/Common+mode+correction+algorithms#Commonmodecorrectionalgorithms-#4-MEDIANalgorithm-detectordependent)

In [ ]:
rawData = det.raw_data(event)
pedSub=rawData-ped
pedSubGain = pedSub*gain

#median method EPIX common mode
calib4 = det.calib(event, cmpars=(4,6,30,10))

#median method with p
calib6 = det.calib(event, cmpars=[6], rms=rms)

pedSubHis = np.histogram(pedSub, np.arange(-10,250))
pedSubGainHis = np.histogram(pedSubGain, np.arange(-10,250))
cm4His = np.histogram(calib4, np.arange(-10,250))
cm6His = np.histogram(calib6, np.arange(-10,250))

img6_thres = calib6.copy()
img6_thres[calib6<15]=0
img6_thres[calib6>=15]=1
img6_drop = ndi.label(img6_thres)

img4_thres = calib4.copy()
img4_thres[calib4<15]=0
img4_thres[calib4>=15]=1
img4_drop = ndi.label(img4_thres)

imgp_thres = pedSub.copy()
imgp_thres[pedSub<15]=0
imgp_thres[pedSub>=15]=1
imgp_drop = ndi.label(imgp_thres)

imgpg_thres = pedSubGain.copy()
imgpg_thres[pedSubGain<15]=0
imgpg_thres[pedSubGain>=15]=1
imgpg_drop = ndi.label(imgpg_thres)

ndrops6 = np.arange(1,img6_drop[1]+1)
ndrops4 = np.arange(1,img4_drop[1]+1)
ndropsp = np.arange(1,imgp_drop[1]+1)
ndropspg = np.arange(1,imgpg_drop[1]+1)

adus6 = ndi.sum(calib6, img6_drop[0], ndrops6)
adus4 = ndi.sum(calib4, img4_drop[0], ndrops4)
adusp = ndi.sum(pedSub, imgp_drop[0], ndropsp)
aduspg = ndi.sum(pedSubGain, imgpg_drop[0], ndropspg)

adu4His = np.histogram(adus4, np.arange(-10,750))
adu6His = np.histogram(adus6, np.arange(-10,750))
adupHis = np.histogram(adusp, np.arange(-10,750))
adupgHis = np.histogram(aduspg, np.arange(-10,750))

This image has no common mode correction. You can see the rows&colums - this way, you cannot tell if this is a common mode or a pedestal problem. Using common mode correction also corrects for sub-ideal pedestals, but during operations it hides that pedestals start to shift. If yuo have pedestal shifts, the common mode is also no longer 'symmetric' around zero as we need to protect from using shifts introduced by signal from moving the pedestal.

In [ ]:
hv_image(pedSubGain)

## Pixel histograms of different treatments.

In [ ]:
hv.Points((pedSubHis[1][1:], np.log(pedSubHis[0])),label='no cm, no gain').options(width=750) *\
hv.Points((pedSubGainHis[1][1:], np.log(pedSubGainHis[0])),label='no cm') *\
hv.Points((cm6His[1][1:], np.log(cm6His[0])),label='cm6') *\
hv.Points((cm4His[1][1:], np.log(cm4His[0])),label='cm4')

### zero-photon peak

In [ ]:
hv.Curve((pedSubHis[1][:40], pedSubHis[0][:40]),label='no cm, no gain').options(width=750) *\
hv.Curve((pedSubGainHis[1][:40], pedSubGainHis[0][:40]),label='no cm') *\
hv.Curve((cm6His[1][:40], cm6His[0][:40]),label='cm6') *\
hv.Curve((cm4His[1][:40], cm4His[0][:40]),label='cm4')

### droplets: one-photon peak, linear scale

In [ ]:
hv.Curve((adupHis[1][:250], adupHis[0][:250]),label='no cm, no gain').options(width=750) *\
hv.Curve((adupgHis[1][:250], adupgHis[0][:250]),label='no cm') *\
hv.Curve((adu6His[1][:250], adu6His[0][:250]),label='cm6') *\
hv.Curve((adu4His[1][:250], adu4His[0][:250]),label='cm4') 

### droplets: log scale

In [ ]:
hv.Curve((adupHis[1][:-1], np.log(adupHis[0])),label='no cm, no gain').options(width=750) *\
hv.Curve((adupgHis[1][:-1], np.log(adupgHis[0])),label='no cm')*\
hv.Curve((adu6His[1][:-1], np.log(adu6His[0])),label='cm6') *\
hv.Curve((adu4His[1][:-1], np.log(adu4His[0])),label='cm4') 